# lo-fAi Inference
this notebook is for running the model and a easy to use runner for model, which heavily depends on the code in the reps the following parts is just installing libs.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg unzip unrar p7zip-full
%pip install torch --upgrade --extra-index-url https://download.pytorch.org/whl/cu121
%pip install pydub tinysoundfont patool mido pyunpack rwkv

here we get the model and the code and sound fonts

In [ ]:
!git lfs install
!git clone https://huggingface.co/soheilsalimidev/lo-fAi  ./models
!git clone https://github.com/soheilsalimidev/lo-fAi.git  ./codes
!mkdir soundFont
from pyunpack import Archive
Archive('./models/soundFonts.rar').extractall('./soundFont/')

here we set the setting of project

In [ ]:
import os
import sys

os.environ["RWKV_JIT_ON"] = "1"
os.environ["RWKV_CUDA_ON"] = "0"
os.environ["LOCAL"] = "0"
os.environ["PIANO_MODEL_PATH"] = './models/rwkv-piano.pth'
os.environ["DRUM_MODEL_PATH"] = './models/rwkv-drum.pth'
os.environ["SOUND_FONT"] = './soundFont'
os.environ["tokenizer"] = './codes/packages/model/src/tokenizer/tokenizer-midi/tokenizer.json'

sys.path.append("./codes/packages/inference")
sys.path.append("./codes/packages/model")
sys.path.append("./codes/packages/model/src/tokenizer")

Here you can set the parameters

- **song_len**
  The length of the song to be generated. The model will generate a song of this length, and if the generated song is longer than song_len, it will be truncated to song_len. If the generated song is shorter than song_len, it will be repeated to match the desired length.

  Default value: 100
- **out_len**
  The final length of the generated song. If the generated song is longer than out_len, it will be truncated to out_len. If the generated song is shorter than out_len, it will not be repeated.

  Default value: 60 seconds

- **temperature**
  The temperature of the model. A lower temperature will result in more predictable and conservative output, while a higher temperature will result in more creative and unpredictable output.

  Default value: 1.0


In [ ]:
song_len = 100
out_len = 60
temperature = 1.0

This section explains how to use the model to generate two songs. The first song will feature the piano as the main instrument, while the second will use a randomly selected instrument.

To guide the model, you can provide a formatted MIDI file input like “2f:f t15 14:5 t100”. This specifies notes, durations, and time shifts. However, even if you leave the input empty, the model will still generate two songs which is random.

Here are some randomly generated song from the model (note for some reason the filling sound are too loud in these exmples but its fix now the final code)

<audio controls>
  <source src="..." type="audio/mpeg">
  Your browser does not support the audio element.
</audio>

<audio controls>
  <source src="..." type="audio/mpeg">
  Your browser does not support the audio element.
</audio>


In [ ]:
from IPython.display import Audio, display
import random
from inferRWKV import mixTheSongs

context = input('Enter encode context for leave empty for random generation')
if context == "":
    context = f"{random.randint(50,90):x}:{random.randint(1,16):x} t{random.randint(20,80)} " * 2

songPath, RandInstrument = mixTheSongs(song_len, context, out_len, temperature)

print('With base piano instruments')
display(Audio(songPath, autoplay=True))
print('With Random instruments')
display(Audio(RandInstrument, autoplay=True))